In [26]:
import os
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
load_dotenv()
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [9]:
with open("hl_podcast_42.txt", "r") as f:
    transcript = f.read()

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500
)

documents = text_splitter.create_documents([transcript])

In [28]:
def chat_gpt_query(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [29]:
for doc in tqdm(documents):
    content = doc.page_content
    prompt = f"""Create 2 lists. In those lists, list all the habits and 
                nutrients mentioned in this text that are beneficial or harmful to brain performance. 
                If there are none mentioned, return: 'Benefitial: -' or 'Harmful: -'. 
                Here's the transcript: {content}"""
    result = chat_gpt_query(prompt)
    print(result)

  5% 1/22 [00:07<02:29,  7.12s/it]

Beneficial habits and nutrients mentioned in the text:
1. Time-restricted feeding or intermittent fasting
2. Eating foods that are good for brain health and focus
3. Metabolically accessible foods that can be converted into energy for the brain
4. Believing in the health benefits and energy-boosting effects of the food

Harmful habits and nutrients mentioned in the text:
None mentioned.

Therefore, the lists would be:
Beneficial: Time-restricted feeding, brain-healthy foods, metabolically accessible foods, belief in food's benefits.
Harmful: -


  9% 2/22 [00:14<02:19,  6.99s/it]

List of habits beneficial to brain performance:
1. Wearing eyeglasses and sunglasses of high quality and clarity.
2. Using lightweight eyeglasses and sunglasses that are comfortable and don't slip off.
3. Taking Athletic Greens, a vitamin mineral probiotic drink that supports brain health through gut-brain access.
4. Practicing meditation with the help of the Headspace app.
5. Engaging in consistent meditation to reduce stress, improve sleep, and enhance cognition.

List of harmful habits to brain performance: -


 14% 3/22 [00:19<02:00,  6.32s/it]

...

Beneficial: 
- Foods that increase blood glucose and brain metabolism
- Foods that are good for brain health
- Foods that promote brain health

Harmful: 
None mentioned in the text.


 95% 21/22 [01:58<00:05,  5.43s/it]

Habits mentioned in the text that are beneficial or harmful to brain performance:

Beneficial habits:
- Pairing foods that are good for brain metabolism with other meals
- Ingesting things that are good for brain health
- Consuming foods and food components that are healthier for the brain

Harmful habits:
- Consuming foods that are highly palatable and sweet, which can shift the dopamine system and lead to cravings for unhealthy foods

Nutrients mentioned in the text that are beneficial or harmful to brain performance:
- No specific nutrients are mentioned in the text.


100% 22/22 [02:02<00:00,  5.56s/it]

Beneficial habits and nutrients mentioned in the text:
- Altering dietary choices over time and in the correct way to reshape food preference and metabolism of the brain
- Consuming superfoods that enhance short and long-term cognition and neuronal health

Harmful habits and nutrients mentioned in the text:
- None mentioned in the given text


In [15]:
type(texts[0])

langchain_core.documents.base.Document

In [ ]:
for chunk in chunks:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Create 2 lists. In those lists, list all the habits and nutrients mentioned in this text that are beneficial or harmful to brain performance. If there are none mentioned, return: 'Benefitial: -' or 'Harmful: -'. Here's the transcript:": {chunk}"}
        ]
    )
    information = extract_information(response['choices'][0]['message']['content'])